# Homework 4 - Spark

In this homework, we are practicing Apache Spark.

You are required to turn in this notebook as BDM\_HW4\_Spark\_**NetId**.ipynb. You will be asked to complete each task using Apache Spark. Output can be printed in the notebook.

## Task 1 (5 points)

You are asked to implement Homework 3 using Spark. The description is provided below for your convenience.

You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [1]:
email_fn = 'enron_mails_small.csv'
email = sc.textFile(email_fn)
list(enumerate(email.first().split(',')))

[(0, 'Date'), (1, 'From'), (2, 'To')]

In [2]:
def extractRelation(partitionId, rows):
    if partitionId == 0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if fields[1].endswith('enron.com'):
            sender = fields[1].split('@')[0].replace('.', ' ').title()
            receiver = [x.split('@')[0].replace('.', ' ').title() for x in 
                        list(filter(lambda x: (x.endswith('@enron.com') and x != fields[0]), fields[2].split(';')))]
            for i in receiver:
                if sender<i:
                    yield ((sender, i), ['0'])
                else:
                    yield ((i, sender), ['1'])


def extractRecipropcal(rows):
    for row in rows:
        if len(row[1]) == 2:
            yield 'reciprocal', row[0][0]+' : '+ row[0][1]


Recipropcal_List = email.mapPartitionsWithIndex(extractRelation)\
    .reduceByKey(lambda x, y: list(set(list(x)+ list(y))))\
    .mapPartitions(extractRecipropcal)

In [3]:
print(len(Recipropcal_List.take(50)))
Recipropcal_List.take(50)

35


[('reciprocal', 'Mark Taylor : Tana Jones'),
 ('reciprocal', 'Mark Haedicke : Mark Taylor'),
 ('reciprocal', 'Brenda Whitehead : Elizabeth Sager'),
 ('reciprocal', 'Carol Clair : Sara Shackleton'),
 ('reciprocal', 'Mark Haedicke : Richard Sanders'),
 ('reciprocal', 'Elizabeth Sager : Janette Elbertson'),
 ('reciprocal', 'Stinson Gibner : Vince Kaminski'),
 ('reciprocal', 'Shirley Crenshaw : Vince Kaminski'),
 ('reciprocal', 'Drew Fossum : Susan Scott'),
 ('reciprocal', 'Michelle Cash : Twanda Sweet'),
 ('reciprocal', 'Fletcher Sturm : Greg Whalley'),
 ('reciprocal', 'Carol Clair : Debra Perlingiere'),
 ('reciprocal', 'Mark Taylor : Sara Shackleton'),
 ('reciprocal', 'Carol Clair : Mark Taylor'),
 ('reciprocal', 'Elizabeth Sager : Mark Taylor'),
 ('reciprocal', 'Elizabeth Sager : Mark Haedicke'),
 ('reciprocal', 'Richard Sanders : Sara Shackleton'),
 ('reciprocal', 'Mark Haedicke : Twanda Sweet'),
 ('reciprocal', 'Sara Shackleton : Tana Jones'),
 ('reciprocal', 'Carol Clair : Tana Jones

## Task 2 (5 points)

You are asked to implement Task 2 of Lab 5. The description is provided below for your convenience.

We’ll be using two NYC open data sets: the SAT Results and the NYC High School Directory data sets. Both can be downloaded from the links below, or from online class resources.

**Dataset**: *Please note that each school is uniquely identified by an DBN code, which should be found on both data sets.*

**SAT_Results.csv**
Source: https://nycopendata.socrata.com/Education/SAT-Results/f9bf-2cp4  
Description: “The most recent school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012.”

**DOE_High_School_Directory_2014-2015.csv**
Source: https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2  
Description: “Directory of NYC High Schools.”

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your task is to compute the average Math scores of all schools along each bus line and subway line. You can find the bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values.

The top ten lines with highest score are shown below.

In [4]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'
Schools = sc.textFile(HSD_FN)

In [5]:
dfScores = spark.read.load(SAT_FN, format = 'csv',
                          header=True, inferSchema = True)

dfScores = dfScores.select('DBN',
                          dfScores['`SAT Math Avg. Score`'].cast('int').alias('score'),
                          dfScores['Num of SAT Test Takers'].cast('int').alias('ntakers')
                          ).na.drop()

dfScores = dfScores.select('DBN',
                          (dfScores.score*dfScores.ntakers).alias('total'),
                          'ntakers')

#dfScores.show()

In [6]:
def extractBus(partitionId, rows):
    if partitionId == 0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields)==58:
            for i in fields[10].split(','):
                yield(fields[0], i.replace(' ', ''))

Bus_List = Schools.mapPartitionsWithIndex(extractBus)
dfBus = spark.createDataFrame(Bus_List, ('dbn','bus')).na.drop()
#dfSchools.show()

In [7]:
def extractSub(partitionId, rows):
    if partitionId == 0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields)==58:
            for x in [i.split(',') for i in fields[11].split(';')]:
                sublist = []
                for y in x:
                    n = y.replace(' ', '')[0]
                    if n not in sublist:
                        yield(fields[0], n)
                        sublist.append(n)

Sub_List = Schools.mapPartitionsWithIndex(extractSub)
dfSub = spark.createDataFrame(Sub_List, ('dbn','sub')).na.drop()

In [8]:
dfResults = dfBus.join(dfScores, dfBus.dbn==dfScores.DBN, how='inner')\
    .groupBy('bus').sum('total', 'ntakers')

dfResults  = dfResults.withColumn('avg', (dfResults[1])/dfResults[2])\
    .select('bus','avg')\
    .orderBy(['avg'],ascending=[0])

bus_rdd = dfResults.rdd.map(tuple).mapValues(lambda x: int(str(x).split('.')[0]))
bus_rdd.take(10)

[('S1115', 612),
 ('M79', 594),
 ('Q42', 582),
 ('M22', 574),
 ('Bx3', 571),
 ('B52', 560),
 ('B63', 557),
 ('B69', 548),
 ('B54', 543),
 ('B25', 541)]

In [9]:
dfResults1 = dfSub.join(dfScores, dfSub.dbn==dfScores.DBN, how='inner')\
    .groupBy('sub').sum('total', 'ntakers')

dfResults1  = dfResults1.withColumn('avg', (dfResults1[1])/dfResults1[2])\
    .select('sub','avg')\
    .orderBy(['avg'],ascending=[0])

sub_rdd = dfResults1.rdd.map(tuple).mapValues(lambda x: int(str(x).split('.')[0]))
sub_rdd.take(10)

[('3', 513),
 ('C', 510),
 ('A', 510),
 ('R', 508),
 ('G', 503),
 ('D', 502),
 ('E', 501),
 ('1', 499),
 ('4', 495),
 ('B', 491)]